In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dataframeTesT').getOrCreate()

In [13]:
columns = ["language","users_count"]
data = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]

In [14]:
rdd = spark.sparkContext.parallelize(data)

In [15]:
dfFromRDD1 = rdd.toDF()

In [16]:
columns = ["language","users_count"]
dfFromRDD1 = rdd.toDF(columns)

In [17]:
dfFromRDD1.printSchema()

root
 |-- language: string (nullable = true)
 |-- users_count: long (nullable = true)



***Convert PySpark Dataframe to Pandas*** <br>
PySpark DataFrame provides a method toPandas() to convert it to Python Pandas DataFrame.<br><br>
**toPandas()** results in the collection of all records in the PySpark DataFrame to the driver program and should be done only on a small subset of the data. running on larger dataset’s results in memory error and crashes the application. To deal with a larger dataset, you can also try increasing memory on the driver.

In [18]:
dfFromRDD1.toPandas()

,language,users_count
0,Java,20000
1,Python,100000
2,Scala,3000


***Access a dataframe column***

In [21]:
# Using DataFrame object (df)
dfFromRDD1.select(dfFromRDD1.users_count).show()
dfFromRDD1.select(dfFromRDD1["users_count"]).show()

#Using SQL col() function
from pyspark.sql.functions import col
dfFromRDD1.select(col("users_count")).show()


+-----------+
|users_count|
+-----------+
|      20000|
|     100000|
|       3000|
+-----------+

+-----------+
|users_count|
+-----------+
|      20000|
|     100000|
|       3000|
+-----------+

+-----------+
|users_count|
+-----------+
|      20000|
|     100000|
|       3000|
+-----------+



In [22]:
data=[("James","Bond","100",None),
      ("Ann","Varsa","200",'F'),
      ("Tom Cruise","XXX","400",''),
      ("Tom Brand",None,"400",'M')] 
columns=["fname","lname","id","gender"]
df=spark.createDataFrame(data,columns)

***alias() – Set’s name to Column***

In [23]:
from pyspark.sql.functions import expr
df.select(df.fname.alias("first_name"), \
          df.lname.alias("last_name")
   ).show()

#Another example
df.select(expr(" fname ||','|| lname").alias("fullName") \
   ).show()

+----------+---------+
|first_name|last_name|
+----------+---------+
|     James|     Bond|
|       Ann|    Varsa|
|Tom Cruise|      XXX|
| Tom Brand|     null|
+----------+---------+

+--------------+
|      fullName|
+--------------+
|    James,Bond|
|     Ann,Varsa|
|Tom Cruise,XXX|
|          null|
+--------------+



***asc() & desc() – Sort the DataFrame columns by Ascending or Descending order***

In [24]:
df.sort(df.fname.asc()).show()
df.sort(df.fname.desc()).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|       Ann|Varsa|200|     F|
|     James| Bond|100|  null|
| Tom Brand| null|400|     M|
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
+----------+-----+---+------+



***cast() & astype() – Used to convert the data Type.***

In [27]:
df.select(df.fname,df.id.cast("int")).printSchema()

root
 |-- fname: string (nullable = true)
 |-- id: integer (nullable = true)



***between() – Returns a Boolean expression when a column values in between lower and upper bound.***

In [28]:
df.filter(df.id.between(100,300)).show()

+-----+-----+---+------+
|fname|lname| id|gender|
+-----+-----+---+------+
|James| Bond|100|  null|
|  Ann|Varsa|200|     F|
+-----+-----+---+------+



***contains() – Checks if a DataFrame column value contains a a value specified in this function.***

In [29]:
df.filter(df.fname.contains("Cruise")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



***startswith() & endswith() – Checks if the value of the DataFrame Column starts and ends with a String respectively.***

In [30]:
df.filter(df.fname.startswith("T")).show()
df.filter(df.fname.endswith("Cruise")).show()

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
| Tom Brand| null|400|     M|
+----------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



***isNull & isNotNull() – Checks if the DataFrame column has NULL or non NULL values.***

In [31]:
df.filter(df.lname.isNull()).show()
df.filter(df.lname.isNotNull()).show()

+---------+-----+---+------+
|    fname|lname| id|gender|
+---------+-----+---+------+
|Tom Brand| null|400|     M|
+---------+-----+---+------+

+----------+-----+---+------+
|     fname|lname| id|gender|
+----------+-----+---+------+
|     James| Bond|100|  null|
|       Ann|Varsa|200|     F|
|Tom Cruise|  XXX|400|      |
+----------+-----+---+------+



***like() & rlike() – Similar to SQL LIKE expression***

In [39]:
df.select(df.fname,df.lname,df.id) \
  .filter(df.fname.like("%om%")).show()

+----------+-----+---+
|     fname|lname| id|
+----------+-----+---+
|Tom Cruise|  XXX|400|
| Tom Brand| null|400|
+----------+-----+---+



***substr() – Returns a Column after getting sub string from the Column***

In [43]:
df.select(df.fname.substr(1,3).alias("substr")).show()

+------+
|substr|
+------+
|   Jam|
|   Ann|
|   Tom|
|   Tom|
+------+



***when() & otherwise() – It is similar to SQL Case When, executes sequence of expressions until it matches the condition and returns a value when match.***

In [44]:
from pyspark.sql.functions import when
df.select(df.fname,df.lname,when(df.gender=="M","Male") \
              .when(df.gender=="F","Female") \
              .when(df.gender==None ,"") \
              .otherwise(df.gender).alias("new_gender") \
    ).show()

+----------+-----+----------+
|     fname|lname|new_gender|
+----------+-----+----------+
|     James| Bond|      null|
|       Ann|Varsa|    Female|
|Tom Cruise|  XXX|          |
| Tom Brand| null|      Male|
+----------+-----+----------+



***isin() – Check if value presents in a List.***

In [45]:
li=["100","200"]
df.select(df.fname,df.lname,df.id) \
  .filter(df.id.isin(li)) \
  .show()

+-----+-----+---+
|fname|lname| id|
+-----+-----+---+
|James| Bond|100|
|  Ann|Varsa|200|
+-----+-----+---+



***getField() – To get the value by key from MapType column and by stuct child name from StructType column***

In [46]:
from pyspark.sql.types import StructType,StructField,StringType,ArrayType,MapType
data=[(("James","Bond"),["Java","C#"],{'hair':'black','eye':'brown'}),
      (("Ann","Varsa"),[".NET","Python"],{'hair':'brown','eye':'black'}),
      (("Tom Cruise",""),["Python","Scala"],{'hair':'red','eye':'grey'}),
      (("Tom Brand",None),["Perl","Ruby"],{'hair':'black','eye':'blue'})]

schema = StructType([
        StructField('name', StructType([
            StructField('fname', StringType(), True),
            StructField('lname', StringType(), True)])),
        StructField('languages', ArrayType(StringType()),True),
        StructField('properties', MapType(StringType(),StringType()),True)
     ])
df=spark.createDataFrame(data,schema)
df.printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- fname: string (nullable = true)
 |    |-- lname: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [47]:
#getField from MapType
df.select(df.properties.getField("hair")).show()

#getField from Struct
df.select(df.name.getField("fname")).show()

+----------------+
|properties[hair]|
+----------------+
|           black|
|           brown|
|             red|
|           black|
+----------------+

+----------+
|name.fname|
+----------+
|     James|
|       Ann|
|Tom Cruise|
| Tom Brand|
+----------+

